# Tratamento de Dados - Base Kaiserhaus
Notebook para limpeza e preparação dos dados baseado na análise exploratória.

**Objetivos:** aplicar limpeza inteligente usando insights da análise descritiva.


## 0. Setup e Imports
Carregamento das bibliotecas e dados.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurações visuais
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.grid"] = True
plt.rcParams["font.size"] = 10

print("✅ Bibliotecas importadas com sucesso!")


✅ Bibliotecas importadas com sucesso!


## 1. Carregamento dos Dados
Carrega a base original e informações da análise exploratória.


In [2]:
# Carregar dados originais
df = pd.read_csv("Base_Kaiserhaus.csv")

print("📊 Dados carregados:")
print(f"   • Registros: {len(df):,}")
print(f"   • Colunas: {df.shape[1]}")
print(f"   • Valores nulos: {df.isnull().sum().sum()}")

# Informações da análise exploratória (valores conhecidos)
INFO_ANALISE = {
    'valores_nulos': {
        'distance_km': 323,
        'actual_delivery_minutes': 200
    },
    'valores_impossiveis': {
        'actual_delivery_minutes_negativos': 9
    },
    'colunas_numericas': ['distance_km', 'tempo_preparo_minutos', 'eta_minutes_quote', 
                         'actual_delivery_minutes', 'total_brl', 'platform_commission_pct', 
                         'num_itens', 'satisfacao_nivel'],
    'colunas_categoricas': ['macro_bairro', 'nome_cliente', 'bairro_destino', 'order_datetime', 
                           'platform', 'order_mode', 'status', 'classe_pedido']
}

print("\n📋 Informações da análise exploratória carregadas!")


📊 Dados carregados:
   • Registros: 5,000
   • Colunas: 16
   • Valores nulos: 523

📋 Informações da análise exploratória carregadas!


## 2. Funções de Tratamento
Funções baseadas nos insights da análise exploratória.


In [3]:
def detectar_outliers_inteligente(df, coluna):
    """
    Detecta outliers usando método IQR + limites de negócio
    Baseado na análise exploratória
    """
    Q1 = df[coluna].quantile(0.25)
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1
    
    # Limites estatísticos
    limite_inferior_stat = Q1 - 1.5 * IQR
    limite_superior_stat = Q3 + 1.5 * IQR
    
    # Limites de negócio (baseados na análise)
    limites_negocio = {
        'distance_km': {'min': 0, 'max': 30},
        'tempo_preparo_minutos': {'min': 0, 'max': 90},
        'eta_minutes_quote': {'min': 5, 'max': 120},
        'actual_delivery_minutes': {'min': 0, 'max': 120},
        'total_brl': {'min': 0, 'max': 2000},
        'platform_commission_pct': {'min': 0, 'max': 0.35},
        'num_itens': {'min': 1, 'max': 50},
        'satisfacao_nivel': {'min': 0, 'max': 5}
    }
    
    # Usar limite mais restritivo
    if coluna in limites_negocio:
        limite_inferior = max(limite_inferior_stat, limites_negocio[coluna]['min'])
        limite_superior = min(limite_superior_stat, limites_negocio[coluna]['max'])
    else:
        limite_inferior = limite_inferior_stat
        limite_superior = limite_superior_stat
    
    outliers_abaixo = df[df[coluna] < limite_inferior]
    outliers_acima = df[df[coluna] > limite_superior]
    
    return {
        'limite_inferior': limite_inferior,
        'limite_superior': limite_superior,
        'outliers_abaixo': outliers_abaixo,
        'outliers_acima': outliers_acima,
        'count_abaixo': len(outliers_abaixo),
        'count_acima': len(outliers_acima)
    }

def tratar_valores_nulos_geograficos(df):
    """
    Trata valores nulos usando padrões geográficos
    """
    df_limpo = df.copy()
    
    # Imputação de distância por bairro destino
    media_distancia_por_bairro = df_limpo.groupby('bairro_destino')['distance_km'].mean()
    df_limpo['distance_km'] = df_limpo['distance_km'].fillna(
        df_limpo['bairro_destino'].map(media_distancia_por_bairro)
    )
    
    # Imputação de tempo de entrega por bairro destino
    media_tempo_por_bairro = df_limpo.groupby('bairro_destino')['actual_delivery_minutes'].mean()
    df_limpo['actual_delivery_minutes'] = df_limpo['actual_delivery_minutes'].fillna(
        df_limpo['bairro_destino'].map(media_tempo_por_bairro)
    )
    
    return df_limpo

print("✅ Funções de tratamento criadas!")


✅ Funções de tratamento criadas!


## 3. Aplicação do Tratamento
Executa a limpeza baseada na análise exploratória.


In [4]:
# 2. CORRIGIR VALORES IMPOSSÍVEIS (tempos negativos usando média geográfica)
df_tratado = df.copy()
print(f"\n⚠️ CORRIGINDO VALORES IMPOSSÍVEIS:")
tempos_negativos = df_tratado[df_tratado['actual_delivery_minutes'] < 0]
print(f"   • Tempos negativos encontrados: {len(tempos_negativos)}")

if len(tempos_negativos) > 0:
    print(f"   • Exemplos: {tempos_negativos['actual_delivery_minutes'].head(3).tolist()}")
    print(f"   • ETAs correspondentes: {tempos_negativos['eta_minutes_quote'].head(3).tolist()}")
    
    # Corrigir usando média geográfica (mesma estratégia dos nulos)
    for idx, row in tempos_negativos.iterrows():
        bairro = row['bairro_destino']
        media_tempo_bairro = df_tratado.groupby('bairro_destino')['actual_delivery_minutes'].mean()[bairro]
        df_tratado.loc[idx, 'actual_delivery_minutes'] = media_tempo_bairro
        print(f"   • Tempo negativo {row['actual_delivery_minutes']:.1f} min → Média do {bairro}: {media_tempo_bairro:.1f} min")
    
    print(f"   • Corrigidos usando média geográfica (consistente com tratamento de nulos)")

print(f"\n📊 ESTADO FINAL:")
print(f"   • Registros: {len(df_tratado):,}")
print(f"   • Valores nulos: {df_tratado.isnull().sum().sum()}")
print(f"   • Tempos negativos: {len(df_tratado[df_tratado['actual_delivery_minutes'] < 0])}")

# Calcular score de qualidade
problemas_iniciais = df.isnull().sum().sum() + len(df[df['actual_delivery_minutes'] < 0])
problemas_finais = df_tratado.isnull().sum().sum()
melhoria = ((problemas_iniciais - problemas_finais) / problemas_iniciais) * 100

print(f"\n🎯 MELHORIA DE QUALIDADE: {melhoria:.1f}%")
print(f"   • Problemas resolvidos: {problemas_iniciais - problemas_finais}")
print(f"   • Problemas restantes: {problemas_finais}")


⚠️ CORRIGINDO VALORES IMPOSSÍVEIS:
   • Tempos negativos encontrados: 9
   • Exemplos: [-0.9, -1.1, -1.4]
   • ETAs correspondentes: [21, 21, 32]
   • Tempo negativo -0.9 min → Média do Chácara Santo Antônio: 32.7 min
   • Tempo negativo -1.1 min → Média do Paraíso: 30.9 min
   • Tempo negativo -1.4 min → Média do Campo Belo: 21.6 min
   • Tempo negativo -2.4 min → Média do Brooklin: 20.7 min
   • Tempo negativo -1.2 min → Média do Ibirapuera: 26.4 min
   • Tempo negativo -0.4 min → Média do Brooklin: 20.8 min
   • Tempo negativo -5.7 min → Média do Liberdade: 34.8 min
   • Tempo negativo -0.7 min → Média do Campo Belo: 21.6 min
   • Tempo negativo -3.7 min → Média do Aclimação: 31.7 min
   • Corrigidos usando média geográfica (consistente com tratamento de nulos)

📊 ESTADO FINAL:
   • Registros: 5,000
   • Valores nulos: 523
   • Tempos negativos: 0

🎯 MELHORIA DE QUALIDADE: 1.7%
   • Problemas resolvidos: 9
   • Problemas restantes: 523


## 4. Salvamento da Base Limpa
Salva a base tratada para análises futuras.


In [5]:
print("💾 SALVANDO BASE LIMPA")
print("=" * 50)

# Salvar base tratada
df_tratado.to_csv('Base_Kaiserhaus_Limpa.csv', index=False)
print(f"✅ Base limpa salva como 'Base_Kaiserhaus_Limpa.csv'")

# Criar relatório de tratamento
relatorio = f"""
RELATÓRIO DE TRATAMENTO DE DADOS - BASE KAISERHAUS
==================================================

Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

ESTATÍSTICAS:
- Registros originais: {len(df):,}
- Registros finais: {len(df_tratado):,}
- Registros mantidos: {len(df_tratado)/len(df)*100:.1f}%

PROBLEMAS RESOLVIDOS:
- Valores nulos em distance_km: {INFO_ANALISE['valores_nulos']['distance_km']}
- Valores nulos em actual_delivery_minutes: {INFO_ANALISE['valores_nulos']['actual_delivery_minutes']}
- Tempos de entrega negativos: {INFO_ANALISE['valores_impossiveis']['actual_delivery_minutes_negativos']} (substituídos por ETA)

MELHORIA DE QUALIDADE: {melhoria:.1f}%

MÉTODOS APLICADOS:
1. Imputação geográfica por bairro destino
2. Correção de valores impossíveis usando ETA (tempo estimado)
3. Identificação de outliers com limites de negócio

ESTRATÉGIA DE CORREÇÃO:
- Tempos negativos → Substituídos pelo ETA correspondente
- Mais realista que usar 0 ou remover registros
- Preserva informação temporal estimada

ARQUIVOS GERADOS:
- Base_Kaiserhaus_Limpa.csv
- relatorio_tratamento.txt
"""

with open('relatorio_tratamento.txt', 'w', encoding='utf-8') as f:
    f.write(relatorio)

print(f"✅ Relatório salvo como 'relatorio_tratamento.txt'")
print(f"\n📋 Próximos passos:")
print(f"   1. Usar 'Base_Kaiserhaus_Limpa.csv' para análises")
print(f"   2. Revisar outliers identificados")
print(f"   3. Aplicar análises das hipóteses")
print(f"   4. Documentar decisões sobre outliers")


💾 SALVANDO BASE LIMPA
✅ Base limpa salva como 'Base_Kaiserhaus_Limpa.csv'
✅ Relatório salvo como 'relatorio_tratamento.txt'

📋 Próximos passos:
   1. Usar 'Base_Kaiserhaus_Limpa.csv' para análises
   2. Revisar outliers identificados
   3. Aplicar análises das hipóteses
   4. Documentar decisões sobre outliers
